In [5]:
import numpy as np 
import os
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import keras
import tensorflow as tf
from keras import backend as K
from keras import metrics
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, InputLayer, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.metrics import AUC
from keras.optimizers import Adam

In [6]:
seed_value= 42

In [7]:
train_path = "C:/Users/Zeel soni/Downloads/archive/chest_xray/train"
val_path =  "C:/Users/Zeel soni/Downloads/archive/chest_xray/val"
test_path = "C:/Users/Zeel soni/Downloads/archive/chest_xray/test"

hyper_dimension = 64
hyper_batch_size = 128
hyper_epochs = 100
hyper_channels = 1
hyper_mode = 'grayscale'

train_datagen = ImageDataGenerator(rescale=1.0/255.0, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2, 
                                   horizontal_flip = True)
val_datagen = ImageDataGenerator(rescale=1.0/255.0) 
test_datagen = ImageDataGenerator(rescale=1.0/255.0) 


train_generator = train_datagen.flow_from_directory(directory = train_path, 
                                                    target_size = (hyper_dimension, hyper_dimension),
                                                    batch_size = hyper_batch_size, 
                                                    color_mode = hyper_mode,
                                                    class_mode = 'binary', 
                                                    seed = 42)
val_generator = val_datagen.flow_from_directory(directory = val_path, 
                                                 target_size = (hyper_dimension, hyper_dimension),
                                                 batch_size = hyper_batch_size, 
                                                 class_mode = 'binary',
                                                 color_mode = hyper_mode,
                                                 shuffle=False,
                                                 seed = 42)
test_generator = test_datagen.flow_from_directory(directory = test_path, 
                                                 target_size = (hyper_dimension, hyper_dimension),
                                                 batch_size = hyper_batch_size, 
                                                 class_mode = 'binary',
                                                 color_mode = hyper_mode,
                                                 shuffle=False,
                                                 seed = 42)

test_generator.reset()

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [8]:
cnn = Sequential()
cnn.add(InputLayer(input_shape=(hyper_dimension, hyper_dimension, hyper_channels)))#Input Layer

cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))#Hidden Layers
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))#Hidden Layers
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))#Hidden Layers
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Flatten())

cnn.add(Dense(activation='relu', units=128))
cnn.add(Dense(activation='sigmoid', units=1))#Output Layer

cnn.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=[AUC()])
cnn_model = cnn.fit(
    train_generator,
    steps_per_epoch=5,
    epochs=25,
    validation_data=val_generator,
    validation_steps=2,
    verbose=1
)


Epoch 1/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 33s 3s/step - auc_1: 0.4601 - loss: 0.6445 - val_auc_1: 0.3359 - val_loss: 0.9693
Epoch 2/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - auc_1: 0.4485 - loss: 0.5860 - val_auc_1: 0.3438 - val_loss: 0.7934
Epoch 3/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - auc_1: 0.4341 - loss: 0.5679 - val_auc_1: 0.3672 - val_loss: 0.8449
Epoch 4/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - auc_1: 0.5257 - loss: 0.5693 - val_auc_1: 0.4219 - val_loss: 0.7569
Epoch 5/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - auc_1: 0.6379 - loss: 0.5587 - val_auc_1: 0.4375 - val_loss: 0.8742
Epoch 6/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - auc_1: 0.6344 - loss: 0.5908 - val_auc_1: 0.6094 - val_loss: 0.7098
Epoch 7/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - auc_1: 0.7289 - loss: 0.5415 - val_auc_1: 0.5938 - val_loss: 0.8248
Epoch 8/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 198ms/step - auc_1: 0.7771 - loss: 0.5255 - val_auc_1: 0.6953 - val_loss: 0.7481
Epoch 9/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - auc_

In [9]:
train_loss = cnn_model.history['loss']
val_loss = cnn_model.history['val_loss']

train_auc = cnn_model.history[list(cnn_model.history.keys())[3]]
val_auc = cnn_model.history[list(cnn_model.history.keys())[1]]

y_true = test_generator.classes
Y_pred = cnn.predict(test_generator, steps=len(test_generator))
y_pred = (Y_pred > 0.5).flatten()
y_pred_prob = Y_pred.flatten()

5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step


In [10]:
## Summary Statistics
cm = confusion_matrix(y_true, y_pred)
TN, FP, FN, TP = cm.ravel()
accuracy = (TP + TN) / np.sum(cm)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
specificity = TN / (TN + FP)
f1 = 2 * precision * recall / (precision + recall)
print(f'[Summary Statistics]\n'
      f'Accuracy = {accuracy:.2%} | Precision = {precision:.2%} | '
      f'Recall = {recall:.2%} | Specificity = {specificity:.2%} | '
      f'F1 Score = {f1:.2%}')

[Summary Statistics]
Accuracy = 84.29% | Precision = 93.20% | Recall = 80.77% | Specificity = 90.17% | F1 Score = 86.54%
